unit-cycleごとの基本統計量をとったデータからモデル作成・予測

In [ ]:
#import zipfile

#with zipfile.ZipFile('drive/MyDrive/engine/train.zip') as existing_zip:
#    existing_zip.extractall('drive/MyDrive/engine')

In [ ]:
#with zipfile.ZipFile('drive/MyDrive/engine/train_stats.zip') as existing_zip:
#    existing_zip.extractall('drive/MyDrive/engine')

In [ ]:
#with zipfile.ZipFile('drive/MyDrive/engine/test.zip') as existing_zip:
#    existing_zip.extractall('drive/MyDrive/engine')

In [ ]:
#with zipfile.ZipFile('drive/MyDrive/engine/test_stats.zip') as existing_zip:
#    existing_zip.extractall('drive/MyDrive/engine')

In [ ]:
!pip install pytorch-tabnet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ご自身のパスを設定
YOUR_PATH = "/drive/MyDrive/engine/"

In [ ]:
cd {YOUR_PATH}

[Errno 2] No such file or directory: '/drive/MyDrive/engine/'
/content


In [ ]:
import os
import gc
import time
import random
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error

sns.set()

In [ ]:
#上限表示数を拡張
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 50)

In [ ]:
def set_seed(seed=200):
  np.random.seed(seed)
  random.seed(seed)
  os.environ["PYTHONHASHSEED"]=str(seed)

ファイルの内容を確認

In [ ]:
input_paths = sorted(glob('./drive/MyDrive/engine/train_stats/*.csv'))
len(input_paths)

9

In [ ]:
df = pd.read_csv(input_paths[0])
print(df.shape)
df.head()

(459, 311)


,unit-cycle,alt_mean,alt_std,alt_min,alt_median,alt_max,alt_first,alt_last,Mach_mean,Mach_std,Mach_min,Mach_median,Mach_max,Mach_first,Mach_last,TRA_mean,TRA_std,TRA_min,TRA_median,TRA_max,TRA_first,TRA_last,T2_mean,T2_std,T2_min,T2_median,T2_max,T2_first,T2_last,T24_mean,T24_std,T24_min,T24_median,T24_max,T24_first,T24_last,T30_mean,T30_std,T30_min,T30_median,T30_max,T30_first,T30_last,T48_mean,T48_std,T48_min,T48_median,T48_max,T48_first,T48_last,T50_mean,T50_std,T50_min,T50_median,T50_max,T50_first,T50_last,P15_mean,P15_std,P15_min,P15_median,P15_max,P15_first,P15_last,P2_mean,P2_std,P2_min,P2_median,P2_max,P2_first,P2_last,P21_mean,P21_std,P21_min,P21_median,P21_max,P21_first,P21_last,P24_mean,P24_std,P24_min,P24_median,P24_max,P24_first,P24_last,Ps30_mean,Ps30_std,Ps30_min,Ps30_median,Ps30_max,Ps30_first,Ps30_last,P40_mean,P40_std,P40_min,P40_median,P40_max,P40_first,P40_last,P50_mean,...,SmHPC_mean,SmHPC_std,SmHPC_min,SmHPC_median,SmHPC_max,SmHPC_first,SmHPC_last,phi_mean,phi_std,phi_min,phi_median,phi_max,phi_first,phi_last,fan_eff_mod_mean,fan_eff_mod_std,fan_eff_mod_min,fan_eff_mod_median,fan_eff_mod_max,fan_eff_mod_first,fan_eff_mod_last,fan_flow_mod_mean,fan_flow_mod_std,fan_flow_mod_min,fan_flow_mod_median,fan_flow_mod_max,fan_flow_mod_first,fan_flow_mod_last,LPC_eff_mod_mean,LPC_eff_mod_std,LPC_eff_mod_min,LPC_eff_mod_median,LPC_eff_mod_max,LPC_eff_mod_first,LPC_eff_mod_last,LPC_flow_mod_mean,LPC_flow_mod_std,LPC_flow_mod_min,LPC_flow_mod_median,LPC_flow_mod_max,LPC_flow_mod_first,LPC_flow_mod_last,HPC_eff_mod_mean,HPC_eff_mod_std,HPC_eff_mod_min,HPC_eff_mod_median,HPC_eff_mod_max,HPC_eff_mod_first,HPC_eff_mod_last,HPC_flow_mod_mean,HPC_flow_mod_std,HPC_flow_mod_min,HPC_flow_mod_median,HPC_flow_mod_max,HPC_flow_mod_first,HPC_flow_mod_last,HPT_eff_mod_mean,HPT_eff_mod_std,HPT_eff_mod_min,HPT_eff_mod_median,HPT_eff_mod_max,HPT_eff_mod_first,HPT_eff_mod_last,HPT_flow_mod_mean,HPT_flow_mod_std,HPT_flow_mod_min,HPT_flow_mod_median,HPT_flow_mod_max,HPT_flow_mod_first,HPT_flow_mod_last,LPT_eff_mod_mean,LPT_eff_mod_std,LPT_eff_mod_min,LPT_eff_mod_median,LPT_eff_mod_max,LPT_eff_mod_first,LPT_eff_mod_last,LPT_flow_mod_mean,LPT_flow_mod_std,LPT_flow_mod_min,LPT_flow_mod_median,LPT_flow_mod_max,LPT_flow_mod_first,LPT_flow_mod_last,Fc_mean,Fc_std,Fc_min,Fc_median,Fc_max,Fc_first,Fc_last,hs_mean,hs_std,hs_min,hs_median,hs_max,hs_first,hs_last,count,Y
0,01-001,8760.0,2431.100522,3004.0,10480.0,10500.0,3012.0,3004.0,0.4736,0.063171,0.2705,0.5220,0.5264,0.3765,0.2722,53.44,15.810174,26.98,58.80,77.94,70.3,26.98,509.5,4.317635,504.2,507.5,522.5,522.5,515.5,585.5,15.426999,558.5,588.5,619.0,618.5,567.5,1355.0,66.717966,1239.0,1372.0,1471.0,1470.0,1266.0,1650.0,125.065138,1426.0,1682.0,1853.0,1850.0,1475.0,1157.0,50.786228,1059.0,1160.0,1270.0,1269.0,1116.0,15.625,1.098637,13.805,15.29,19.44,19.44,16.16,12.380,0.755769,11.766,11.920,14.484,14.484,13.820,15.87,1.115399,14.010,15.520,19.73,19.73,16.40,19.05,1.738928,16.05,18.84,24.44,24.40,18.47,276.8,49.087161,197.9,280.8,395.0,394.8,228.4,282.0,49.510191,202.4,285.8,401.5,401.2,233.4,12.450,...,28.94,2.330255,25.30,28.33,33.40,25.64,32.20,37.90,2.869917,32.56,38.70,42.25,42.16,33.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000604,0.0,-0.000604,-0.000604,-0.000604,-0.000604,-0.000604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,4498,99
1,01-002,9890.0,2984.519602,3004.0,10664.0,12750.0,3012.0,3004.0,0.4844,0.065859,0.3293,0.5156,0.5540,0.3433,0.3298,52.16,15.806911,26.28,56.78,76.56,76.1,26.36,506.5,5.639289,500.2,505.5,521.0,520.0,519.0,580.5,15.512961,552.5,579.5,621.0,621.0,570.0,1341.0,66.727322,1221.0,1350.0,1490.0,1489.0,1266.0,1628.0,124.817650,1406.0,1651.0,1891.0,1889.0,1470.0,1143.0,51.136217,1055.0,1138.0,1290.0,1289.0,1112.0,15.000,1.268298,13.360,14.5

Y(RUL)は、unitごとに1つの値に定まるものですが、与えられたデータは、unit1のcycle1ではYは99、cycle2では98、cycle3では97...という形式となっています。<br>
今回はこの形式のまま、Yを予測するタスクとして解いてみます。

全ファイルを加工・統合

In [ ]:
# n cycle先のunitcycle名を取得する関数を作成

def get_future_unitcycle(unitcycle, unitcycle_dict, ncycle=1):
    """
    unitcycle（unitが1でcycleが1なら'01-001'、unitが1でcycleが2なら'01-002'、といった値）について、ncycle分先のunitcycleを返す
    """
    unit = unitcycle.split('-')[0]
    if unit in unitcycle_dict:
        if unitcycle in unitcycle_dict[unit]:
            idx = unitcycle_dict[unit].index(unitcycle)
            future_idx = idx+ncycle
            if 0 <= future_idx < len(unitcycle_dict[unit]):
                return unitcycle_dict[unit][future_idx]
            else:
                # print(f'ncycle {ncycle} is out of list')
                return
        else:
            # print(f'unit {unit} found, but unitcyle {unitcycle} not found')
            return
    else:
        # print(f'unit {unit} not found')
        return


# 動作確認

unitcycle_list = df['unit-cycle'].unique()
unit_list = sorted(list(set([i.split('-')[0] for i in unitcycle_list])))

unitcycle_dict = {unit:[] for unit in unit_list}
for unitcycle in unitcycle_list:
    key = unitcycle.split('-')[0]
    unitcycle_dict[key].append(unitcycle)

print(unitcycle_dict['01'])
get_future_unitcycle('01-001', unitcycle_dict, ncycle=2)

['01-001', '01-002', '01-003', '01-004', '01-005', '01-006', '01-007', '01-008', '01-009', '01-010', '01-011', '01-012', '01-013', '01-014', '01-015', '01-016', '01-017', '01-018', '01-019', '01-020', '01-021', '01-022', '01-023', '01-024', '01-025', '01-026', '01-027', '01-028', '01-029', '01-030', '01-031', '01-032', '01-033', '01-034', '01-035', '01-036', '01-037', '01-038', '01-039', '01-040', '01-041', '01-042', '01-043', '01-044', '01-045', '01-046', '01-047', '01-048', '01-049', '01-050', '01-051', '01-052', '01-053', '01-054', '01-055', '01-056', '01-057', '01-058', '01-059', '01-060', '01-061', '01-062', '01-063', '01-064', '01-065', '01-066', '01-067', '01-068', '01-069', '01-070', '01-071', '01-072', '01-073', '01-074', '01-075', '01-076', '01-077', '01-078', '01-079', '01-080', '01-081', '01-082', '01-083', '01-084', '01-085', '01-086', '01-087', '01-088', '01-089', '01-090', '01-091', '01-092', '01-093', '01-094', '01-095', '01-096', '01-097', '01-098', '01-099', '01-100']

'01-003'

In [ ]:
# 説明変数として何cycle前から何cycle前の値を使うかを決める

newer_use_cycle = 1 # 説明変数として何cycle前の値から使うか（ここで指定した値のcycleは含む）
older_use_cycle = 4 # 説明変数として何cycle前の値まで使うか（ここで指定した値のcycleは含まない）

target = 'Y'
id = 'unit-cycle'
drop_features = ['filename']

In [ ]:
def process_concat_data(input_paths):

    dfs = []

    for path in input_paths:
        
        df = pd.read_csv(path)
        filename = path.split('/')[-1].split('_')[1]
        df['filename'] = filename

        unitcycle_list = df[id].unique()
        unit_list = sorted(list(set([i.split('-')[0] for i in unitcycle_list])))

        unitcycle_dict = {unit:[] for unit in unit_list}
        for unitcycle in unitcycle_list:
            key = unitcycle.split('-')[0]
            unitcycle_dict[key].append(unitcycle)
        
        features = [col for col in df.columns if col not in [id, target]+drop_features]
        feat_df = df[[id]+features]
        if 'train' in path:
            df = df[[id, target] + drop_features]
        else:
            df = df[[id] + drop_features]

        for i in range(newer_use_cycle, older_use_cycle): # 説明変数として使えるcycleの値をここで指定
            tmp_df = feat_df.copy()
            tmp_df[id] = tmp_df[id].apply(lambda x: get_future_unitcycle(x, unitcycle_dict, i))
            tmp_df.columns = [f'{col}_prev{i}' if col != id else col for col in tmp_df.columns]
            df = pd.merge(df, tmp_df, on=id, how='left') # 後方の値で欠損値補完
            df = df.astype("object").fillna(method='bfill')

        df['cycle'] = df[id].apply(lambda x: int(x.split('-')[1]))
        df['unit'] = df[id].apply(lambda x: x.split('-')[0])
        df['filename-unit'] = df['filename'] + '-' + df['unit']
        df['unit'] = df['unit'].map(int)

        dfs.append(df)

    df = pd.concat(dfs)
    df.reset_index(drop=True, inplace=True)
    return df

In [ ]:
train_paths = sorted(glob('./drive/MyDrive/engine/train_stats/*.csv'))
df = process_concat_data(train_paths)
test_paths = sorted(glob('./drive/MyDrive/engine/test_stats/*.csv'))
test_df = process_concat_data(test_paths)
df.shape, test_df.shape

((3844, 933), (1920, 932))

In [ ]:
sub_df = pd.read_csv('./drive/MyDrive/engine/sample_submission.csv')

In [ ]:
#df['LPT_e_cycle'] = df['LPT_eff_mod_mean_prev3']/df['cycle']
#df['LPT_f_cycle'] = df['LPT_flow_mod_mean_prev3']/df['cycle']
#df['HPT_e_cycle'] = df['HPT_eff_mod_mean_prev3']/df['cycle']

df['hs_cycle'] = df['hs_mean_prev3']/df['cycle']
df['SmHPC_phi'] = df['SmHPC_mean_prev3']/df['phi_mean_prev3']

#グラフわけ
df['Alt_Mach'] = df['alt_mean_prev3']*df['Mach_mean_prev3']
df['graph_A'] = df['TRA_mean_prev3']*df['Nf_mean_prev3']*df['Nc_mean_prev3']*df['T40_mean_prev3']*df['phi_mean_prev3']
df['graph_B'] = df['T24_mean_prev3']*df['T30_mean_prev3']*df['T48_mean_prev3']*df['T50_mean_prev3']
df['graph_C'] = df['T2_mean_prev3']*df['P15_mean_prev3']*df['P2_mean_prev3']*df['P21_mean_prev3']*df['P24_mean_prev3']*df['Ps30_mean_prev3']*df['P40_mean_prev3']*df['P50_mean_prev3']*df['Wf_mean_prev3']*df['P30_mean_prev3']*df['P45_mean_prev3']*df['W21_mean_prev3']*df['W22_mean_prev3']*df['W25_mean_prev3']*df['W31_mean_prev3']*df['W32_mean_prev3']*df['W48_mean_prev3']*df['W50_mean_prev3']*df['SmLPC_mean_prev3']

df['margin'] = df['SmFan_mean_prev3']+df['SmLPC_mean_prev3']+df['SmHPC_mean_prev3']
df['margin_HPT'] = df['margin']/df['T48_mean_prev3']
df['health_mh'] = df['hs_mean_prev3']/df['margin_HPT']

df['LPT_eff_flow_HPT_eff'] = df['LPT_eff_mod_mean_prev3']*df['LPT_flow_mod_mean_prev3']*df['HPT_eff_mod_mean_prev3']

df['fan'] = df['fan_eff_mod_mean_prev3']*df['fan_flow_mod_mean_prev3']
df['LPC'] = df['LPC_eff_mod_mean_prev3']*df['LPC_flow_mod_mean_prev3']
df['HPC'] = df['HPC_eff_mod_mean_prev3']*df['HPC_flow_mod_mean_prev3']
df['HPT'] = df['HPT_eff_mod_mean_prev3']*df['HPT_flow_mod_mean_prev3']
df['LPT'] = df['LPT_eff_mod_mean_prev3']*df['LPT_flow_mod_mean_prev3']
df['champss'] = df['fan']*df['LPC']*df['HPC']*df['HPT']*df['LPT']


df['lowpress_temperature'] = df['T24_mean_prev3']-df['T2_mean_prev3'] 
df['high_lowpress_temperature'] = df['T30_mean_prev3']-df['T24_mean_prev3']
df['nensho_temperature'] = df['T48_mean_prev3']-df['T30_mean_prev3']
df['haishutu_temperature'] = df['T50_mean_prev3']-df['T48_mean_prev3']

df['fan_pressure'] = df['P2_mean_prev3']*df['P21_mean_prev3']
df['press_pressure'] = df['P24_mean_prev3']*df['Ps30_mean_prev3']
df['axis_pressure'] = df['Ps30_mean_prev3']*df['P15_mean_prev3']*df['P50_mean_prev3']
df['nensho_pressure'] = df['P15_mean_prev3']*df['P50_mean_prev3']
df['kaiten_pressure'] = df['fan_pressure']*df['press_pressure']
df['bocho_pressure'] = df['nensho_pressure']/df['kaiten_pressure']
df['out_pressure'] = df['P40_mean_prev3']/df['P50_mean_prev3']

'''
df['2_W22'] = df['W22_mean_prev3']**2
df['2_W25'] = df['W25_mean_prev3']**2
df['2_W31'] = df['W31_mean_prev3']**2
df['2_W48'] = df['W48_mean_prev3']**2
df['2_W50'] = df['W50_mean_prev3']**2

#out_flowはW21とW22で単位が違うので、ちょっと違う意味だけど、(W22+out_flow)*t = W21として考えた。そのため、わり算ぐらいしかない。
#df['out_flow'] = df['W21_mean_prev3']/df['W22_mean_prev3']

df['press_flow'] = df['W25_mean_prev3']*df['W22_mean_prev3']
df['nensho_flow'] = df['W31_mean_prev3']*df['W25_mean_prev3']
df['HPT_cb'] = df['W48_mean_prev3']*df['W31_mean_prev3']
df['LPT_cb'] = df['W50_mean_prev3']*df['W32_mean_prev3']

#mixはあくまでoutとLPT_cbが合流していることを意味するだけ
df['mix_flow'] = df['out_flow']*df['LPT_cb']

#df['EPR'] = df['P50_mean_prev3']/df['P24_mean_prev3']



df['kaitensa'] = df['Nf_mean_prev3']-df['Nc_mean_prev3']
df['air_kaiten'] = df['kaitensa']/df['W21_mean_prev3']

df['fan_sadouline'] = df['W21_mean_prev3']/df['P2_mean_prev3']*df['fan_eff_mod_mean_prev3']*df['Nf_mean_prev3']
df['fan_stole'] = df['fan_sadouline']/df['SmFan_mean_prev3']

df['core_sadouline'] = df['HPT_cb']/df['P50_mean_prev3']*df['LPT_eff_mod_mean_prev3']*df['Nc_mean_prev3']
df['core_stole'] = df['core_sadouline']/df['SmFan_mean_prev3']

df['kakudo_kouritsu'] = df['TRA_mean_prev3']/df['W21_mean_prev3']

#ジェットターボエンジンの推力の公式
df['kasoku'] = df['LPT_cb'] - df['Mach_mean_prev3']
df['air_sokudo'] = df['W21_mean_prev3']/32.2*df['kasoku']
df['turbo_sokudo'] = df['Wf_mean_prev3']/32.2*df['LPT_cb']
df['thruster'] = df['air_sokudo'] + df['turbo_sokudo'] + df['P50_mean_prev3']


from sklearn.cluster import KMeans
np_df = np.array(df['SmLPC_mean_prev3'])
np_df = np_df.reshape(-1,1)
df['c_kmean'] = KMeans(n_clusters=3).fit_predict(np_df)

'''

"\ndf['2_W22'] = df['W22_mean_prev3']**2\ndf['2_W25'] = df['W25_mean_prev3']**2\ndf['2_W31'] = df['W31_mean_prev3']**2\ndf['2_W48'] = df['W48_mean_prev3']**2\ndf['2_W50'] = df['W50_mean_prev3']**2\n\n#out_flowはW21とW22で単位が違うので、ちょっと違う意味だけど、(W22+out_flow)*t = W21として考えた。そのため、わり算ぐらいしかない。\n#df['out_flow'] = df['W21_mean_prev3']/df['W22_mean_prev3']\n\ndf['press_flow'] = df['W25_mean_prev3']*df['W22_mean_prev3']\ndf['nensho_flow'] = df['W31_mean_prev3']*df['W25_mean_prev3']\ndf['HPT_cb'] = df['W48_mean_prev3']*df['W31_mean_prev3']\ndf['LPT_cb'] = df['W50_mean_prev3']*df['W32_mean_prev3']\n\n#mixはあくまでoutとLPT_cbが合流していることを意味するだけ\ndf['mix_flow'] = df['out_flow']*df['LPT_cb']\n\n#df['EPR'] = df['P50_mean_prev3']/df['P24_mean_prev3']\n\n\n\ndf['kaitensa'] = df['Nf_mean_prev3']-df['Nc_mean_prev3']\ndf['air_kaiten'] = df['kaitensa']/df['W21_mean_prev3']\n\ndf['fan_sadouline'] = df['W21_mean_prev3']/df['P2_mean_prev3']*df['fan_eff_mod_mean_prev3']*df['Nf_mean_prev3']\ndf['fan_stole'] = df['fan_sadouli

In [ ]:
df.head()

,unit-cycle,Y,filename,alt_mean_prev1,alt_std_prev1,alt_min_prev1,alt_median_prev1,alt_max_prev1,alt_first_prev1,alt_last_prev1,Mach_mean_prev1,Mach_std_prev1,Mach_min_prev1,Mach_median_prev1,Mach_max_prev1,Mach_first_prev1,Mach_last_prev1,TRA_mean_prev1,TRA_std_prev1,TRA_min_prev1,TRA_median_prev1,TRA_max_prev1,TRA_first_prev1,TRA_last_prev1,T2_mean_prev1,T2_std_prev1,T2_min_prev1,T2_median_prev1,T2_max_prev1,T2_first_prev1,T2_last_prev1,T24_mean_prev1,T24_std_prev1,T24_min_prev1,T24_median_prev1,T24_max_prev1,T24_first_prev1,T24_last_prev1,T30_mean_prev1,T30_std_prev1,T30_min_prev1,T30_median_prev1,T30_max_prev1,T30_first_prev1,T30_last_prev1,T48_mean_prev1,T48_std_prev1,T48_min_prev1,T48_median_prev1,T48_max_prev1,T48_first_prev1,T48_last_prev1,T50_mean_prev1,T50_std_prev1,T50_min_prev1,T50_median_prev1,T50_max_prev1,T50_first_prev1,T50_last_prev1,P15_mean_prev1,P15_std_prev1,P15_min_prev1,P15_median_prev1,P15_max_prev1,P15_first_prev1,P15_last_prev1,P2_mean_prev1,P2_std_prev1,P2_min_prev1,P2_median_prev1,P2_max_prev1,P2_first_prev1,P2_last_prev1,P21_mean_prev1,P21_std_prev1,P21_min_prev1,P21_median_prev1,P21_max_prev1,P21_first_prev1,P21_last_prev1,P24_mean_prev1,P24_std_prev1,P24_min_prev1,P24_median_prev1,P24_max_prev1,P24_first_prev1,P24_last_prev1,Ps30_mean_prev1,Ps30_std_prev1,Ps30_min_prev1,Ps30_median_prev1,Ps30_max_prev1,Ps30_first_prev1,Ps30_last_prev1,P40_mean_prev1,P40_std_prev1,P40_min_prev1,P40_median_prev1,P40_max_prev1,P40_first_prev1,...,LPC_eff_mod_std_prev3,LPC_eff_mod_min_prev3,LPC_eff_mod_median_prev3,LPC_eff_mod_max_prev3,LPC_eff_mod_first_prev3,LPC_eff_mod_last_prev3,LPC_flow_mod_mean_prev3,LPC_flow_mod_std_prev3,LPC_flow_mod_min_prev3,LPC_flow_mod_median_prev3,LPC_flow_mod_max_prev3,LPC_flow_mod_first_prev3,LPC_flow_mod_last_prev3,HPC_eff_mod_mean_prev3,HPC_eff_mod_std_prev3,HPC_eff_mod_min_prev3,HPC_eff_mod_median_prev3,HPC_eff_mod_max_prev3,HPC_eff_mod_first_prev3,HPC_eff_mod_last_prev3,HPC_flow_mod_mean_prev3,HPC_flow_mod_std_prev3,HPC_flow_mod_min_prev3,HPC_flow_mod_median_prev3,HPC_flow_mod_max_prev3,HPC_flow_mod_first_prev3,HPC_flow_mod_last_prev3,HPT_eff_mod_mean_prev3,HPT_eff_mod_std_prev3,HPT_eff_mod_min_prev3,HPT_eff_mod_median_prev3,HPT_eff_mod_max_prev3,HPT_eff_mod_first_prev3,HPT_eff_mod_last_prev3,HPT_flow_mod_mean_prev3,HPT_flow_mod_std_prev3,HPT_flow_mod_min_prev3,HPT_flow_mod_median_prev3,HPT_flow_mod_max_prev3,HPT_flow_mod_first_prev3,HPT_flow_mod_last_prev3,LPT_eff_mod_mean_prev3,LPT_eff_mod_std_prev3,LPT_eff_mod_min_prev3,LPT_eff_mod_median_prev3,LPT_eff_mod_max_prev3,LPT_eff_mod_first_prev3,LPT_eff_mod_last_prev3,LPT_flow_mod_mean_prev3,LPT_flow_mod_std_prev3,LPT_flow_mod_min_prev3,LPT_flow_mod_median_prev3,LPT_flow_mod_max_prev3,LPT_flow_mod_first_prev3,LPT_flow_mod_last_prev3,Fc_mean_prev3,Fc_std_prev3,Fc_min_prev3,Fc_median_prev3,Fc_max_prev3,Fc_first_prev3,Fc_last_prev3,hs_mean_prev3,hs_std_prev3,hs_min_prev3,hs_median_prev3,hs_max_prev3,hs_first_prev3,hs_last_prev3,count_prev3,cycle,unit,filename-unit,hs_cycle,SmHPC_phi,Alt_Mach,graph_A,graph_B,graph_C,margin,margin_HPT,health_mh,LPT_eff_flow_HPT_eff,fan,LPC,HPC,HPT,LPT,champss,lowpress_temperature,high_lowpress_temperature,nensho_temperature,haishutu_temperature,fan_pressure,press_pressure,axis_pressure,nensho_pressure,kaiten_pressure,bocho_pressure,out_pressure
0,01-001,99,DS01-005,8760.0,2431.100522,3004.0,10480.0,10500.0,3012.0,3004.0,0.4736,0.063171,0.2705,0.5220,0.5264,0.3765,0.2722,53.44,15.810174,26.98,58.80,77.94,70.3,26.98,509.5,4.317635,504.2,507.5,522.5,522.5,515.5,585.5,15.426999,558.5,588.5,619.0,618.5,567.5,1355.0,66.717966,1239.0,1372.0,1471.0,1470.0,1266.0,1650.0,125.065138,1426.0,1682.0,1853.0,1850.0,1475.0,1157.0,50.786228,1059.0,1160.0,1270.0,1269.0,1116.0,15.625,1.098637,13.805,15.29,19.44,19.44,16.16,12.380,0.755769,11.766,11.920,14.484,14.484,13.820,15.87,1.115399,14.01,15.520,19.73,19.73,16.40,19.05,1.738928,16.05,18.84,24.44,24.40,18.47,276.8,49.087161,197.9,280.8,395.0,394.8,228.4,282.0,49.510191,20

In [ ]:
#def get_filenum(x):
#  return int(x[2:4])

In [ ]:
#df['filenum'] = df['filename-unit'].apply(get_filenum)

In [ ]:
#def get_unit(x):
#  return int(x[-2:])

In [ ]:
#df['unitnum'] = df['filename-unit'].apply(get_unit)

In [ ]:
'''
df['seijo'] = 0
df['seijo'][(df['filenum'] == 1)&(df['unitnum'] == 1)] = 1
df['seijo'][(df['filenum'] == 1)&(df['unitnum'] == 3)] = 1
df['seijo'][(df['filenum'] == 4)&(df['unitnum'] == 3)] = 1
df['seijo'][(df['filenum'] == 4)&(df['unitnum'] == 5)] = 1
df['seijo'][(df['filenum'] == 5)&(df['unitnum'] == 5)] = 1
'''

"\ndf['seijo'] = 0\ndf['seijo'][(df['filenum'] == 1)&(df['unitnum'] == 1)] = 1\ndf['seijo'][(df['filenum'] == 1)&(df['unitnum'] == 3)] = 1\ndf['seijo'][(df['filenum'] == 4)&(df['unitnum'] == 3)] = 1\ndf['seijo'][(df['filenum'] == 4)&(df['unitnum'] == 5)] = 1\ndf['seijo'][(df['filenum'] == 5)&(df['unitnum'] == 5)] = 1\n"

In [ ]:
print(df)

     unit-cycle   Y   filename  alt_mean_prev1  alt_std_prev1  alt_min_prev1  \
0        01-001  99   DS01-005          8760.0    2431.100522         3004.0   
1        01-002  98   DS01-005          8760.0    2431.100522         3004.0   
2        01-003  97   DS01-005          9890.0    2984.519602         3004.0   
3        01-004  96   DS01-005          7580.0    2240.489704         3002.0   
4        01-005  95   DS01-005          8076.0    3068.155480         3000.0   
...         ...  ..        ...             ...            ...            ...   
3839     05-046   4  DS08c-008         17800.0    7038.849411         3000.0   
3840     05-047   3  DS08c-008         20370.0    8372.404699         3000.0   
3841     05-048   2  DS08c-008         13640.0    4305.348544         3000.0   
3842     05-049   1  DS08c-008         19840.0    7584.433888         3000.0   
3843     05-050   0  DS08c-008         15580.0    5941.385743         3004.0   

      alt_median_prev1  alt_max_prev1  

In [ ]:
'''
df['alt_roll'] = df['alt_mean_prev3'].rolling(10).sum()
df['Mach_roll'] = df['Mach_mean_prev3'].rolling(10).sum()
df['TRA_roll'] = df['TRA_mean_prev3'].rolling(10).sum()
df['T2_roll'] = df['T2_mean_prev3'].rolling(10).sum()
df['T24_roll'] = df['T24_mean_prev3'].rolling(10).sum()
df['T30_roll'] = df['T30_mean_prev3'].rolling(10).sum()
df['T48_roll'] = df['T48_mean_prev3'].rolling(10).sum()
df['T50_roll'] = df['T50_mean_prev3'].rolling(10).sum()
df['P15_roll'] = df['P15_mean_prev3'].rolling(10).sum()
df['P2_roll'] = df['P2_mean_prev3'].rolling(10).sum()
df['P21_roll'] = df['P21_mean_prev3'].rolling(10).sum()
df['P24_roll'] = df['P24_mean_prev3'].rolling(10).sum()
df['Ps30_roll'] = df['Ps30_mean_prev3'].rolling(10).sum()
df['P40_roll']  = df['P40_mean_prev3'].rolling(10).sum()
df['P50_roll'] = df['P50_mean_prev3'].rolling(10).sum()
df['Nf_roll'] = df['Nf_mean_prev3'].rolling(10).sum()
df['Nc_roll'] = df['Nc_mean_prev3'].rolling(10).sum()
df['Wf_roll'] = df['Wf_mean_prev3'].rolling(10).sum()
df['T40_roll'] = df['T40_mean_prev3'].rolling(10).sum()
df['P30_roll'] = df['P30_mean_prev3'].rolling(10).sum()
df['P45_roll'] = df['P45_mean_prev3'].rolling(10).sum()
df['W21_roll'] = df['W21_mean_prev3'].rolling(10).sum()
df['W22_roll'] = df['W22_mean_prev3'].rolling(10).sum()
df['W25_roll'] = df['W25_mean_prev3'].rolling(10).sum()
df['W31_roll'] = df['W31_mean_prev3'].rolling(10).sum()
df['W32_roll'] = df['W32_mean_prev3'].rolling(10).sum()
df['W48_roll'] = df['W48_mean_prev3'].rolling(10).sum()
df['W50_roll'] = df['W50_mean_prev3'].rolling(10).sum()
df['SmFan_roll'] = df['SmFan_mean_prev3'].rolling(10).sum()
df['SmLPC_roll'] = df['SmLPC_mean_prev3'].rolling(10).sum()
df['SmHPC_roll'] = df['SmHPC_mean_prev3'].rolling(10).sum()
df['phi_roll'] = df['phi_mean_prev3'].rolling(10).sum()
df['fan_eff_roll'] = df['fan_eff_mod_mean_prev3'].rolling(10).sum()
df['fan_flow_roll'] = df['fan_flow_mod_mean_prev3'].rolling(10).sum()
df['LPC_eff_roll'] = df['LPC_eff_mod_mean_prev3'].rolling(10).sum()
df['LPC_flow_roll'] = df['LPC_flow_mod_mean_prev3'].rolling(10).sum()
df['HPC_eff_roll'] = df['HPC_eff_mod_mean_prev3'].rolling(10).sum()
df['HPC_flow_roll'] = df['HPC_flow_mod_mean_prev3'].rolling(10).sum()
df['HPT_eff_roll'] = df['HPT_eff_mod_mean_prev3'].rolling(10).sum()
df['HPT_flow_roll'] = df['HPT_flow_mod_mean_prev3'].rolling(10).sum()
df['LPT_eff_roll'] = df['LPT_eff_mod_mean_prev3'].rolling(10).sum()
df['LPT_flow_roll'] = df['LPT_flow_mod_mean_prev3'].rolling(10).sum()
df['Fc_roll'] = df['Fc_mean_prev3'].rolling(10).sum()
df['hs_roll'] = df['hs_mean_prev3'].rolling(10).sum()
'''

"\ndf['alt_roll'] = df['alt_mean_prev3'].rolling(10).sum()\ndf['Mach_roll'] = df['Mach_mean_prev3'].rolling(10).sum()\ndf['TRA_roll'] = df['TRA_mean_prev3'].rolling(10).sum()\ndf['T2_roll'] = df['T2_mean_prev3'].rolling(10).sum()\ndf['T24_roll'] = df['T24_mean_prev3'].rolling(10).sum()\ndf['T30_roll'] = df['T30_mean_prev3'].rolling(10).sum()\ndf['T48_roll'] = df['T48_mean_prev3'].rolling(10).sum()\ndf['T50_roll'] = df['T50_mean_prev3'].rolling(10).sum()\ndf['P15_roll'] = df['P15_mean_prev3'].rolling(10).sum()\ndf['P2_roll'] = df['P2_mean_prev3'].rolling(10).sum()\ndf['P21_roll'] = df['P21_mean_prev3'].rolling(10).sum()\ndf['P24_roll'] = df['P24_mean_prev3'].rolling(10).sum()\ndf['Ps30_roll'] = df['Ps30_mean_prev3'].rolling(10).sum()\ndf['P40_roll']  = df['P40_mean_prev3'].rolling(10).sum()\ndf['P50_roll'] = df['P50_mean_prev3'].rolling(10).sum()\ndf['Nf_roll'] = df['Nf_mean_prev3'].rolling(10).sum()\ndf['Nc_roll'] = df['Nc_mean_prev3'].rolling(10).sum()\ndf['Wf_roll'] = df['Wf_mean_pre

In [ ]:
'''
df['alt_shift'] = df['alt_mean_prev3'].shift(1)
df['Mach_shift'] = df['Mach_mean_prev3'].shift(1)
df['TRA_shift'] = df['TRA_mean_prev3'].shift(1)
df['T2_shift'] = df['T2_mean_prev3'].shift(1)
df['T24_shift'] = df['T24_mean_prev3'].shift(1)
df['T30_shift'] = df['T30_mean_prev3'].shift(1)
df['T48_shift'] = df['T48_mean_prev3'].shift(1)
df['T50_shift'] = df['T50_mean_prev3'].shift(1)
df['P15_shift'] = df['P15_mean_prev3'].shift(1)
df['P2_shift'] = df['P2_mean_prev3'].shift(1)
df['P21_shift'] = df['P21_mean_prev3'].shift(1)
df['P24_shift'] = df['P24_mean_prev3'].shift(1)
df['Ps30_shift'] = df['Ps30_mean_prev3'].shift(1)
df['P40_shift']  = df['P40_mean_prev3'].shift(1)
df['P50_shift'] = df['P50_mean_prev3'].shift(1)
df['Nf_shift'] = df['Nf_mean_prev3'].shift(1)
df['Nc_shift'] = df['Nc_mean_prev3'].shift(1)
df['Wf_shift'] = df['Wf_mean_prev3'].shift(1)
df['T40_shift'] = df['T40_mean_prev3'].shift(1)
df['P30_shift'] = df['P30_mean_prev3'].shift(1)
df['P45_shift'] = df['P45_mean_prev3'].shift(1)
df['W21_shift'] = df['W21_mean_prev3'].shift(1)
df['W22_shift'] = df['W22_mean_prev3'].shift(1)
df['W25_shift'] = df['W25_mean_prev3'].shift(1)
df['W31_shift'] = df['W31_mean_prev3'].shift(1)
df['W32_shift'] = df['W32_mean_prev3'].shift(1)
df['W48_shift'] = df['W48_mean_prev3'].shift(1)
df['W50_shift'] = df['W50_mean_prev3'].shift(1)
df['SmFan_shift'] = df['SmFan_mean_prev3'].shift(1)
df['SmLPC_shift'] = df['SmLPC_mean_prev3'].shift(1)
df['SmHPC_shift'] = df['SmHPC_mean_prev3'].shift(1)
df['phi_shift'] = df['phi_mean_prev3'].shift(1)
df['fan_eff_shift'] = df['fan_eff_mod_mean_prev3'].shift(1)
df['fan_flow_shift'] = df['fan_flow_mod_mean_prev3'].shift(1)
df['LPC_eff_shift'] = df['LPC_eff_mod_mean_prev3'].shift(1)
df['LPC_flow_shift'] = df['LPC_flow_mod_mean_prev3'].shift(1)
df['HPC_eff_shift'] = df['HPC_eff_mod_mean_prev3'].shift(1)
df['HPC_flow_shift'] = df['HPC_flow_mod_mean_prev3'].shift(1)
df['HPT_eff_shift'] = df['HPT_eff_mod_mean_prev3'].shift(1)
df['HPT_flow_shift'] = df['HPT_flow_mod_mean_prev3'].shift(1)
df['LPT_eff_shift'] = df['LPT_eff_mod_mean_prev3'].shift(1)
df['LPT_flow_shift'] = df['LPT_flow_mod_mean_prev3'].shift(1)
df['Fc_shift'] = df['Fc_mean_prev3'].shift(1)
df['hs_shift'] = df['hs_mean_prev3'].shift(1)
'''

"\ndf['alt_shift'] = df['alt_mean_prev3'].shift(1)\ndf['Mach_shift'] = df['Mach_mean_prev3'].shift(1)\ndf['TRA_shift'] = df['TRA_mean_prev3'].shift(1)\ndf['T2_shift'] = df['T2_mean_prev3'].shift(1)\ndf['T24_shift'] = df['T24_mean_prev3'].shift(1)\ndf['T30_shift'] = df['T30_mean_prev3'].shift(1)\ndf['T48_shift'] = df['T48_mean_prev3'].shift(1)\ndf['T50_shift'] = df['T50_mean_prev3'].shift(1)\ndf['P15_shift'] = df['P15_mean_prev3'].shift(1)\ndf['P2_shift'] = df['P2_mean_prev3'].shift(1)\ndf['P21_shift'] = df['P21_mean_prev3'].shift(1)\ndf['P24_shift'] = df['P24_mean_prev3'].shift(1)\ndf['Ps30_shift'] = df['Ps30_mean_prev3'].shift(1)\ndf['P40_shift']  = df['P40_mean_prev3'].shift(1)\ndf['P50_shift'] = df['P50_mean_prev3'].shift(1)\ndf['Nf_shift'] = df['Nf_mean_prev3'].shift(1)\ndf['Nc_shift'] = df['Nc_mean_prev3'].shift(1)\ndf['Wf_shift'] = df['Wf_mean_prev3'].shift(1)\ndf['T40_shift'] = df['T40_mean_prev3'].shift(1)\ndf['P30_shift'] = df['P30_mean_prev3'].shift(1)\ndf['P45_shift'] = df['

In [ ]:
df['alt_minmax'] = df['alt_max_prev3'] - df['alt_min_prev3']
df['Mach_minmax'] = df['Mach_max_prev3'] - df['Mach_min_prev3']
df['TRA_minmax'] = df['TRA_max_prev3'] - df['TRA_min_prev3']
df['T2_minmax'] = df['T2_max_prev3'] - df['T2_min_prev3']
df['T24_minmax'] = df['T24_max_prev3'] - df['T24_min_prev3']
df['T30_minmax'] = df['T30_max_prev3'] - df['T30_min_prev3']
df['T48_minmax'] = df['T48_max_prev3'] - df['T48_min_prev3']
df['T50_minmax'] = df['T50_max_prev3'] - df['T50_min_prev3']
df['P15_minmax'] = df['P15_max_prev3'] - df['P15_min_prev3']
df['P2_minmax'] = df['P2_max_prev3'] - df['P2_min_prev3']
df['P21_minmax'] = df['P21_max_prev3'] - df['P21_min_prev3']
df['P24_minmax'] = df['P24_max_prev3'] - df['P24_min_prev3']
df['Ps30_minmax'] = df['Ps30_max_prev3'] - df['Ps30_min_prev3']
df['P40_minmax'] = df['P40_max_prev3'] - df['P40_min_prev3']
df['P50_minmax'] = df['P50_max_prev3'] - df['P50_min_prev3']
df['Nf_minmax'] = df['Nf_max_prev3'] - df['Nf_min_prev3']
df['Nc_minmax'] = df['Nc_max_prev3'] - df['Nc_min_prev3']
df['Wf_minmax'] = df['Wf_max_prev3'] - df['Wf_min_prev3']
df['T40_minmax'] = df['T40_max_prev3'] - df['T40_min_prev3']
df['P30_minmax'] = df['P30_max_prev3'] - df['P30_min_prev3']
df['P45_minmax'] = df['P45_max_prev3'] - df['P45_min_prev3']
df['W21_minmax'] = df['W21_max_prev3'] - df['W21_min_prev3']
df['W22_minmax'] = df['W22_max_prev3'] - df['W22_min_prev3']
df['W25_minmax'] = df['W25_max_prev3'] - df['W25_min_prev3']
df['W31_minmax'] = df['W31_max_prev3'] - df['W31_min_prev3']
df['W32_minmax'] = df['W32_max_prev3'] - df['W32_min_prev3']
df['W48_minmax'] = df['W48_max_prev3'] - df['W48_min_prev3']
df['W50_minmax'] = df['W50_max_prev3'] - df['W50_min_prev3']
df['SmFan_minmax'] = df['SmFan_max_prev3'] - df['SmFan_min_prev3']
df['SmLPC_minmax'] = df['SmLPC_max_prev3'] - df['SmLPC_min_prev3']
df['SmHPC_minmax'] = df['SmHPC_max_prev3'] - df['SmHPC_min_prev3']
df['phi_minmax'] = df['phi_max_prev3'] - df['phi_min_prev3']
df['fan_eff_minmax'] = df['fan_eff_mod_max_prev3'] - df['fan_eff_mod_min_prev3']
df['fan_flow_minmax'] = df['fan_flow_mod_max_prev3'] - df['fan_flow_mod_min_prev3']
df['LPC_eff_minmax'] = df['LPC_eff_mod_max_prev3'] - df['LPC_eff_mod_min_prev3']
df['LPC_flow_minmax'] = df['LPC_flow_mod_max_prev3'] - df['LPC_flow_mod_min_prev3']
df['HPC_eff_minmax'] = df['HPC_eff_mod_max_prev3'] - df['HPC_eff_mod_min_prev3']
df['HPC_flow_minmax'] = df['HPC_flow_mod_max_prev3'] - df['HPC_flow_mod_min_prev3']
df['HPT_eff_minmax'] = df['HPT_eff_mod_max_prev3'] - df['HPT_eff_mod_min_prev3']
df['HPT_flow_minmax'] = df['HPT_flow_mod_max_prev3'] - df['HPT_flow_mod_min_prev3']
df['LPT_eff_minmax'] = df['LPT_eff_mod_max_prev3'] - df['LPT_eff_mod_min_prev3']
df['LPT_flow_minmax'] = df['LPT_flow_mod_max_prev3'] - df['LPT_flow_mod_min_prev3']
df['Fc_minmax'] = df['Fc_max_prev3'] - df['Fc_min_prev3']
df['hs_minmax'] = df['hs_max_prev3'] - df['hs_min_prev3']

In [ ]:
#df.fillna(method = 'bfill')

In [ ]:
df.head()

,unit-cycle,Y,filename,alt_mean_prev1,alt_std_prev1,alt_min_prev1,alt_median_prev1,alt_max_prev1,alt_first_prev1,alt_last_prev1,Mach_mean_prev1,Mach_std_prev1,Mach_min_prev1,Mach_median_prev1,Mach_max_prev1,Mach_first_prev1,Mach_last_prev1,TRA_mean_prev1,TRA_std_prev1,TRA_min_prev1,TRA_median_prev1,TRA_max_prev1,TRA_first_prev1,TRA_last_prev1,T2_mean_prev1,T2_std_prev1,T2_min_prev1,T2_median_prev1,T2_max_prev1,T2_first_prev1,T2_last_prev1,T24_mean_prev1,T24_std_prev1,T24_min_prev1,T24_median_prev1,T24_max_prev1,T24_first_prev1,T24_last_prev1,T30_mean_prev1,T30_std_prev1,T30_min_prev1,T30_median_prev1,T30_max_prev1,T30_first_prev1,T30_last_prev1,T48_mean_prev1,T48_std_prev1,T48_min_prev1,T48_median_prev1,T48_max_prev1,T48_first_prev1,T48_last_prev1,T50_mean_prev1,T50_std_prev1,T50_min_prev1,T50_median_prev1,T50_max_prev1,T50_first_prev1,T50_last_prev1,P15_mean_prev1,P15_std_prev1,P15_min_prev1,P15_median_prev1,P15_max_prev1,P15_first_prev1,P15_last_prev1,P2_mean_prev1,P2_std_prev1,P2_min_prev1,P2_median_prev1,P2_max_prev1,P2_first_prev1,P2_last_prev1,P21_mean_prev1,P21_std_prev1,P21_min_prev1,P21_median_prev1,P21_max_prev1,P21_first_prev1,P21_last_prev1,P24_mean_prev1,P24_std_prev1,P24_min_prev1,P24_median_prev1,P24_max_prev1,P24_first_prev1,P24_last_prev1,Ps30_mean_prev1,Ps30_std_prev1,Ps30_min_prev1,Ps30_median_prev1,Ps30_max_prev1,Ps30_first_prev1,Ps30_last_prev1,P40_mean_prev1,P40_std_prev1,P40_min_prev1,P40_median_prev1,P40_max_prev1,P40_first_prev1,...,LPT_eff_mod_median_prev3,LPT_eff_mod_max_prev3,LPT_eff_mod_first_prev3,LPT_eff_mod_last_prev3,LPT_flow_mod_mean_prev3,LPT_flow_mod_std_prev3,LPT_flow_mod_min_prev3,LPT_flow_mod_median_prev3,LPT_flow_mod_max_prev3,LPT_flow_mod_first_prev3,LPT_flow_mod_last_prev3,Fc_mean_prev3,Fc_std_prev3,Fc_min_prev3,Fc_median_prev3,Fc_max_prev3,Fc_first_prev3,Fc_last_prev3,hs_mean_prev3,hs_std_prev3,hs_min_prev3,hs_median_prev3,hs_max_prev3,hs_first_prev3,hs_last_prev3,count_prev3,cycle,unit,filename-unit,hs_cycle,SmHPC_phi,Alt_Mach,graph_A,graph_B,graph_C,margin,margin_HPT,health_mh,LPT_eff_flow_HPT_eff,fan,LPC,HPC,HPT,LPT,champss,lowpress_temperature,high_lowpress_temperature,nensho_temperature,haishutu_temperature,fan_pressure,press_pressure,axis_pressure,nensho_pressure,kaiten_pressure,bocho_pressure,out_pressure,alt_minmax,Mach_minmax,TRA_minmax,T2_minmax,T24_minmax,T30_minmax,T48_minmax,T50_minmax,P15_minmax,P2_minmax,P21_minmax,P24_minmax,Ps30_minmax,P40_minmax,P50_minmax,Nf_minmax,Nc_minmax,Wf_minmax,T40_minmax,P30_minmax,P45_minmax,W21_minmax,W22_minmax,W25_minmax,W31_minmax,W32_minmax,W48_minmax,W50_minmax,SmFan_minmax,SmLPC_minmax,SmHPC_minmax,phi_minmax,fan_eff_minmax,fan_flow_minmax,LPC_eff_minmax,LPC_flow_minmax,HPC_eff_minmax,HPC_flow_minmax,HPT_eff_minmax,HPT_flow_minmax,LPT_eff_minmax,LPT_flow_minmax,Fc_minmax,hs_minmax
0,01-001,99,DS01-005,8760.0,2431.100522,3004.0,10480.0,10500.0,3012.0,3004.0,0.4736,0.063171,0.2705,0.5220,0.5264,0.3765,0.2722,53.44,15.810174,26.98,58.80,77.94,70.3,26.98,509.5,4.317635,504.2,507.5,522.5,522.5,515.5,585.5,15.426999,558.5,588.5,619.0,618.5,567.5,1355.0,66.717966,1239.0,1372.0,1471.0,1470.0,1266.0,1650.0,125.065138,1426.0,1682.0,1853.0,1850.0,1475.0,1157.0,50.786228,1059.0,1160.0,1270.0,1269.0,1116.0,15.625,1.098637,13.805,15.29,19.44,19.44,16.16,12.380,0.755769,11.766,11.920,14.484,14.484,13.820,15.87,1.115399,14.01,15.520,19.73,19.73,16.40,19.05,1.738928,16.05,18.84,24.44,24.40,18.47,276.8,49.087161,197.9,280.8,395.0,394.8,228.4,282.0,49.510191,202.4,285.8,401.5,401.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,4498.0,1,1,DS01-005-01,1.000000,0.763588,4148.736,8.280469e+13,1.514550e+12,1.216403e+34,55.514,0.033645,29.722232,-0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,76.0,769.5,295.0,-493.0,196.47060,5273.040,53846.25,194.53125,1.035997e+06,0.000188,22.650602,7496.0,0.2559,50.96,18.3,60.5,232.0,427.0,211.0,5.635,2.718,5.72,8.39,197.1,199.1,5.04,581.0,772.0,2.840,620.0,208.2,37.85,903.0,114.0,

In [ ]:
print(df['hs_mean_prev3'].value_counts())

0.0    2544
1.0    1300
Name: hs_mean_prev3, dtype: int64


In [ ]:
print(df['LPT_eff_mod_mean_prev1'].value_counts())

 0.000000    1965
-0.000795       6
-0.000499       5
-0.001037       4
-0.003330       4
             ... 
-0.001121       1
-0.025700       1
-0.002129       1
-0.002167       1
-0.002647       1
Name: LPT_eff_mod_mean_prev1, Length: 1428, dtype: int64


In [ ]:
#df = df[df['seijo']==1]
print(df)

     unit-cycle   Y   filename  alt_mean_prev1  alt_std_prev1  alt_min_prev1  \
0        01-001  99   DS01-005          8760.0    2431.100522         3004.0   
1        01-002  98   DS01-005          8760.0    2431.100522         3004.0   
2        01-003  97   DS01-005          9890.0    2984.519602         3004.0   
3        01-004  96   DS01-005          7580.0    2240.489704         3002.0   
4        01-005  95   DS01-005          8076.0    3068.155480         3000.0   
...         ...  ..        ...             ...            ...            ...   
3839     05-046   4  DS08c-008         17800.0    7038.849411         3000.0   
3840     05-047   3  DS08c-008         20370.0    8372.404699         3000.0   
3841     05-048   2  DS08c-008         13640.0    4305.348544         3000.0   
3842     05-049   1  DS08c-008         19840.0    7584.433888         3000.0   
3843     05-050   0  DS08c-008         15580.0    5941.385743         3004.0   

      alt_median_prev1  alt_max_prev1  

In [ ]:
#test_df['LPT_e_cycle'] = test_df['LPT_eff_mod_mean_prev3']/test_df['cycle']
#test_df['LPT_f_cycle'] = test_df['LPT_flow_mod_mean_prev3']/test_df['cycle']
#test_df['HPT_e_cycle'] = test_df['HPT_eff_mod_mean_prev3']/test_df['cycle']

test_df['hs_cycle'] = test_df['hs_mean_prev3']/test_df['cycle']
test_df['SmHPC_phi'] = test_df['SmHPC_mean_prev3']/test_df['phi_mean_prev3']

test_df['Alt_Mach'] = test_df['alt_mean_prev3']*test_df['Mach_mean_prev3']
test_df['graph_A'] = test_df['TRA_mean_prev3']*test_df['Nf_mean_prev3']*test_df['Nc_mean_prev3']*test_df['T40_mean_prev3']*test_df['phi_mean_prev3']
test_df['graph_B'] = test_df['T24_mean_prev3']*test_df['T30_mean_prev3']*test_df['T48_mean_prev3']*test_df['T50_mean_prev3']
test_df['graph_C'] = test_df['T2_mean_prev3']*test_df['P15_mean_prev3']*test_df['P2_mean_prev3']*test_df['P21_mean_prev3']*test_df['P24_mean_prev3']*test_df['Ps30_mean_prev3']*test_df['P40_mean_prev3']*test_df['P50_mean_prev3']*test_df['Wf_mean_prev3']*test_df['P30_mean_prev3']*test_df['P45_mean_prev3']*test_df['W21_mean_prev3']*test_df['W22_mean_prev3']*test_df['W25_mean_prev3']*test_df['W31_mean_prev3']*test_df['W32_mean_prev3']*test_df['W48_mean_prev3']*test_df['W50_mean_prev3']*test_df['SmLPC_mean_prev3']

test_df['margin'] = test_df['SmFan_mean_prev3']+test_df['SmLPC_mean_prev3']+test_df['SmHPC_mean_prev3']
test_df['margin_HPT'] = test_df['margin']/test_df['T48_mean_prev3']
test_df['health_mh'] = test_df['hs_mean_prev3']/test_df['margin_HPT']

test_df['LPT_eff_flow_HPT_eff'] = test_df['LPT_eff_mod_mean_prev3']*test_df['LPT_flow_mod_mean_prev3']*test_df['HPT_eff_mod_mean_prev3']

test_df['fan'] = test_df['fan_eff_mod_mean_prev3']*test_df['fan_flow_mod_mean_prev3']
test_df['LPC'] = test_df['LPC_eff_mod_mean_prev3']*test_df['LPC_flow_mod_mean_prev3']
test_df['HPC'] = test_df['HPC_eff_mod_mean_prev3']*test_df['HPC_flow_mod_mean_prev3']
test_df['HPT'] = test_df['HPT_eff_mod_mean_prev3']*test_df['HPT_flow_mod_mean_prev3']
test_df['LPT'] = test_df['LPT_eff_mod_mean_prev3']*test_df['LPT_flow_mod_mean_prev3']
test_df['champss'] = test_df['fan']*test_df['LPC']*test_df['HPC']*test_df['HPT']*test_df['LPT']

test_df['lowpress_temperature'] = test_df['T24_mean_prev3']-test_df['T2_mean_prev3'] 
test_df['high_lowpress_temperature'] = test_df['T30_mean_prev3']-test_df['T24_mean_prev3']
test_df['nensho_temperature'] = test_df['T48_mean_prev3']-test_df['T30_mean_prev3']
test_df['haishutu_temperature'] = test_df['T50_mean_prev3']-test_df['T48_mean_prev3']

test_df['fan_pressure'] = test_df['P2_mean_prev3']*test_df['P21_mean_prev3']
test_df['press_pressure'] = test_df['P24_mean_prev3']*test_df['Ps30_mean_prev3']
test_df['axis_pressure'] = test_df['Ps30_mean_prev3']*test_df['P15_mean_prev3']*test_df['P50_mean_prev3']
test_df['nensho_pressure'] = test_df['P15_mean_prev3']*test_df['P50_mean_prev3']
test_df['kaiten_pressure'] = test_df['fan_pressure']*test_df['press_pressure']
test_df['bocho_pressure'] = test_df['nensho_pressure']/test_df['kaiten_pressure']
test_df['out_pressure'] = test_df['P40_mean_prev3']/test_df['P50_mean_prev3']

'''
test_df['2_W22'] = test_df['W22_mean_prev3']**2
test_df['2_W25'] = test_df['W25_mean_prev3']**2
test_df['2_W31'] = test_df['W31_mean_prev3']**2
test_df['2_W48'] = test_df['W48_mean_prev3']**2
test_df['2_W50'] = test_df['W50_mean_prev3']**2


#out_flowはW21とW22で単位が違うので、ちょっと違う意味だけど、(W22+out_flow)*t = W21として考えた。そのため、わり算ぐらいしかない。
#test_df['out_flow'] = test_df['W21_mean_prev3']/test_df['W22_mean_prev3']

test_df['press_flow'] = test_df['W25_mean_prev3']*test_df['W22_mean_prev3']
test_df['nensho_flow'] = test_df['W31_mean_prev3']*test_df['W25_mean_prev3']
test_df['HPT_cb'] = test_df['W48_mean_prev3']*test_df['W31_mean_prev3']
test_df['LPT_cb'] = test_df['W50_mean_prev3']*test_df['W32_mean_prev3']

#mixはあくまでoutとLPT_cbが合流していることを意味するだけ
test_df['mix_flow'] = test_df['out_flow']*test_df['LPT_cb']

#test_df['EPR'] = test_df['P50_mean_prev3']/test_df['P24_mean_prev3']


test_df['kaitensa'] = test_df['Nf_mean_prev3']-test_df['Nc_mean_prev3']
test_df['air_kaiten'] = test_df['kaitensa']/test_df['W21_mean_prev3']

test_df['fan_sadouline'] = test_df['W21_mean_prev3']/test_df['P2_mean_prev3']*test_df['fan_eff_mod_mean_prev3']*test_df['Nf_mean_prev3']
test_df['fan_stole'] = test_df['fan_sadouline']/test_df['SmFan_mean_prev3']

test_df['core_sadouline'] = test_df['HPT_cb']/test_df['P50_mean_prev3']*test_df['LPT_eff_mod_mean_prev3']*test_df['Nc_mean_prev3']
test_df['core_stole'] = test_df['core_sadouline']/test_df['SmFan_mean_prev3']

test_df['kakudo_kouritsu'] = test_df['TRA_mean_prev3']/test_df['W21_mean_prev3']

test_df['kasoku'] = test_df['LPT_cb'] - test_df['Mach_mean_prev3']
test_df['air_sokudo'] = test_df['W21_mean_prev3']/32.2*test_df['kasoku']
test_df['turbo_sokudo'] = test_df['Wf_mean_prev3']/32.2*test_df['LPT_cb']
test_df['thruster'] = test_df['air_sokudo'] + test_df['turbo_sokudo'] + test_df['P50_mean_prev3']


np_test_df = np.array(test_df['SmLPC_mean_prev3'])
np_test_df = np_test_df.reshape(-1,1)
test_df['c_kmean'] = KMeans(n_clusters=3).fit_predict(np_test_df)
'''


"\ntest_df['2_W22'] = test_df['W22_mean_prev3']**2\ntest_df['2_W25'] = test_df['W25_mean_prev3']**2\ntest_df['2_W31'] = test_df['W31_mean_prev3']**2\ntest_df['2_W48'] = test_df['W48_mean_prev3']**2\ntest_df['2_W50'] = test_df['W50_mean_prev3']**2\n\n\n#out_flowはW21とW22で単位が違うので、ちょっと違う意味だけど、(W22+out_flow)*t = W21として考えた。そのため、わり算ぐらいしかない。\n#test_df['out_flow'] = test_df['W21_mean_prev3']/test_df['W22_mean_prev3']\n\ntest_df['press_flow'] = test_df['W25_mean_prev3']*test_df['W22_mean_prev3']\ntest_df['nensho_flow'] = test_df['W31_mean_prev3']*test_df['W25_mean_prev3']\ntest_df['HPT_cb'] = test_df['W48_mean_prev3']*test_df['W31_mean_prev3']\ntest_df['LPT_cb'] = test_df['W50_mean_prev3']*test_df['W32_mean_prev3']\n\n#mixはあくまでoutとLPT_cbが合流していることを意味するだけ\ntest_df['mix_flow'] = test_df['out_flow']*test_df['LPT_cb']\n\n#test_df['EPR'] = test_df['P50_mean_prev3']/test_df['P24_mean_prev3']\n\n\ntest_df['kaitensa'] = test_df['Nf_mean_prev3']-test_df['Nc_mean_prev3']\ntest_df['air_kaiten'] = test_df['k

In [ ]:
'''
test_df['alt_roll'] = test_df['alt_mean_prev3'].rolling(10).sum()
test_df['Mach_roll'] = test_df['Mach_mean_prev3'].rolling(10).sum()
test_df['TRA_roll'] = test_df['TRA_mean_prev3'].rolling(10).sum()
test_df['T2_roll'] = test_df['T2_mean_prev3'].rolling(10).sum()
test_df['T24_roll'] = test_df['T24_mean_prev3'].rolling(10).sum()
test_df['T30_roll'] = test_df['T30_mean_prev3'].rolling(10).sum()
test_df['T48_roll'] = test_df['T48_mean_prev3'].rolling(10).sum()
test_df['T50_roll'] = test_df['T50_mean_prev3'].rolling(10).sum()
test_df['P15_roll'] = test_df['P15_mean_prev3'].rolling(10).sum()
test_df['P2_roll'] = test_df['P2_mean_prev3'].rolling(10).sum()
test_df['P21_roll'] = test_df['P21_mean_prev3'].rolling(10).sum()
test_df['P24_roll'] = test_df['P24_mean_prev3'].rolling(10).sum()
test_df['Ps30_roll'] = test_df['Ps30_mean_prev3'].rolling(10).sum()
test_df['P40_roll']  = test_df['P40_mean_prev3'].rolling(10).sum()
test_df['P50_roll'] = test_df['P50_mean_prev3'].rolling(10).sum()
test_df['Nf_roll'] = test_df['Nf_mean_prev3'].rolling(10).sum()
test_df['Nc_roll'] =test_df['Nc_mean_prev3'].rolling(10).sum()
test_df['Wf_roll'] = test_df['Wf_mean_prev3'].rolling(10).sum()
test_df['T40_roll'] = test_df['T40_mean_prev3'].rolling(10).sum()
test_df['P30_roll'] = test_df['P30_mean_prev3'].rolling(10).sum()
test_df['P45_roll'] = test_df['P45_mean_prev3'].rolling(10).sum()
test_df['W21_roll'] = test_df['W21_mean_prev3'].rolling(10).sum()
test_df['W22_roll'] = test_df['W22_mean_prev3'].rolling(10).sum()
test_df['W25_roll'] = test_df['W25_mean_prev3'].rolling(10).sum()
test_df['W31_roll'] = test_df['W31_mean_prev3'].rolling(10).sum()
test_df['W32_roll'] = test_df['W32_mean_prev3'].rolling(10).sum()
test_df['W48_roll'] = test_df['W48_mean_prev3'].rolling(10).sum()
test_df['W50_roll'] = test_df['W50_mean_prev3'].rolling(10).sum()
test_df['SmFan_roll'] = test_df['SmFan_mean_prev3'].rolling(10).sum()
test_df['SmLPC_roll'] = test_df['SmLPC_mean_prev3'].rolling(10).sum()
test_df['SmHPC_roll'] = test_df['SmHPC_mean_prev3'].rolling(10).sum()
test_df['phi_roll'] = test_df['phi_mean_prev3'].rolling(10).sum()
test_df['fan_eff_roll'] = test_df['fan_eff_mod_mean_prev3'].rolling(10).sum()
test_df['fan_flow_roll'] = test_df['fan_flow_mod_mean_prev3'].rolling(10).sum()
test_df['LPC_eff_roll'] = test_df['LPC_eff_mod_mean_prev3'].rolling(10).sum()
test_df['LPC_flow_roll'] = test_df['LPC_flow_mod_mean_prev3'].rolling(10).sum()
test_df['HPC_eff_roll'] = test_df['HPC_eff_mod_mean_prev3'].rolling(10).sum()
test_df['HPC_flow_roll'] = test_df['HPC_flow_mod_mean_prev3'].rolling(10).sum()
test_df['HPT_eff_roll'] = test_df['HPT_eff_mod_mean_prev3'].rolling(10).sum()
test_df['HPT_flow_roll'] = test_df['HPT_flow_mod_mean_prev3'].rolling(10).sum()
test_df['LPT_eff_roll'] = test_df['LPT_eff_mod_mean_prev3'].rolling(10).sum()
test_df['LPT_flow_roll'] = test_df['LPT_flow_mod_mean_prev3'].rolling(10).sum()
test_df['Fc_roll'] =test_df['Fc_mean_prev3'].rolling(10).sum()
test_df['hs_roll'] = test_df['hs_mean_prev3'].rolling(10).sum()
'''

"\ntest_df['alt_roll'] = test_df['alt_mean_prev3'].rolling(10).sum()\ntest_df['Mach_roll'] = test_df['Mach_mean_prev3'].rolling(10).sum()\ntest_df['TRA_roll'] = test_df['TRA_mean_prev3'].rolling(10).sum()\ntest_df['T2_roll'] = test_df['T2_mean_prev3'].rolling(10).sum()\ntest_df['T24_roll'] = test_df['T24_mean_prev3'].rolling(10).sum()\ntest_df['T30_roll'] = test_df['T30_mean_prev3'].rolling(10).sum()\ntest_df['T48_roll'] = test_df['T48_mean_prev3'].rolling(10).sum()\ntest_df['T50_roll'] = test_df['T50_mean_prev3'].rolling(10).sum()\ntest_df['P15_roll'] = test_df['P15_mean_prev3'].rolling(10).sum()\ntest_df['P2_roll'] = test_df['P2_mean_prev3'].rolling(10).sum()\ntest_df['P21_roll'] = test_df['P21_mean_prev3'].rolling(10).sum()\ntest_df['P24_roll'] = test_df['P24_mean_prev3'].rolling(10).sum()\ntest_df['Ps30_roll'] = test_df['Ps30_mean_prev3'].rolling(10).sum()\ntest_df['P40_roll']  = test_df['P40_mean_prev3'].rolling(10).sum()\ntest_df['P50_roll'] = test_df['P50_mean_prev3'].rolling(10

In [ ]:
'''
test_df['alt_shift'] = test_df['alt_mean_prev3'].shift(1)
test_df['Mach_shift'] = test_df['Mach_mean_prev3'].shift(1)
test_df['TRA_shift'] = test_df['TRA_mean_prev3'].shift(1)
test_df['T2_shift'] = test_df['T2_mean_prev3'].shift(1)
test_df['T24_shift'] = test_df['T24_mean_prev3'].shift(1)
test_df['T30_shift'] = test_df['T30_mean_prev3'].shift(1)
test_df['T48_shift'] = test_df['T48_mean_prev3'].shift(1)
test_df['T50_shift'] = test_df['T50_mean_prev3'].shift(1)
test_df['P15_shift'] = test_df['P15_mean_prev3'].shift(1)
test_df['P2_shift'] = test_df['P2_mean_prev3'].shift(1)
test_df['P21_shift'] = test_df['P21_mean_prev3'].shift(1)
test_df['P24_shift'] = test_df['P24_mean_prev3'].shift(1)
test_df['Ps30_shift'] = test_df['Ps30_mean_prev3'].shift(1)
test_df['P40_shift']  = test_df['P40_mean_prev3'].shift(1)
test_df['P50_shift'] = test_df['P50_mean_prev3'].shift(1)
test_df['Nf_shift'] = test_df['Nf_mean_prev3'].shift(1)
test_df['Nc_shift'] = test_df['Nc_mean_prev3'].shift(1)
test_df['Wf_shift'] = test_df['Wf_mean_prev3'].shift(1)
test_df['T40_shift'] = test_df['T40_mean_prev3'].shift(1)
test_df['P30_shift'] = test_df['P30_mean_prev3'].shift(1)
test_df['P45_shift'] = test_df['P45_mean_prev3'].shift(1)
test_df['W21_shift'] = test_df['W21_mean_prev3'].shift(1)
test_df['W22_shift'] = test_df['W22_mean_prev3'].shift(1)
test_df['W25_shift'] = test_df['W25_mean_prev3'].shift(1)
test_df['W31_shift'] = test_df['W31_mean_prev3'].shift(1)
test_df['W32_shift'] = test_df['W32_mean_prev3'].shift(1)
test_df['W48_shift'] = test_df['W48_mean_prev3'].shift(1)
test_df['W50_shift'] = test_df['W50_mean_prev3'].shift(1)
test_df['SmFan_shift'] = test_df['SmFan_mean_prev3'].shift(1)
test_df['SmLPC_shift'] = test_df['SmLPC_mean_prev3'].shift(1)
test_df['SmHPC_shift'] = test_df['SmHPC_mean_prev3'].shift(1)
test_df['phi_shift'] = test_df['phi_mean_prev3'].shift(1)
test_df['fan_eff_shift'] = test_df['fan_eff_mod_mean_prev3'].shift(1)
test_df['fan_flow_shift'] = test_df['fan_flow_mod_mean_prev3'].shift(1)
test_df['LPC_eff_shift'] = test_df['LPC_eff_mod_mean_prev3'].shift(1)
test_df['LPC_flow_shift'] = test_df['LPC_flow_mod_mean_prev3'].shift(1)
test_df['HPC_eff_shift'] = test_df['HPC_eff_mod_mean_prev3'].shift(1)
test_df['HPC_flow_shift'] = test_df['HPC_flow_mod_mean_prev3'].shift(1)
test_df['HPT_eff_shift'] = test_df['HPT_eff_mod_mean_prev3'].shift(1)
test_df['HPT_flow_shift'] = test_df['HPT_flow_mod_mean_prev3'].shift(1)
test_df['LPT_eff_shift'] = test_df['LPT_eff_mod_mean_prev3'].shift(1)
test_df['LPT_flow_shift'] = test_df['LPT_flow_mod_mean_prev3'].shift(1)
test_df['Fc_shift'] = test_df['Fc_mean_prev3'].shift(1)
test_df['hs_shift'] = test_df['hs_mean_prev3'].shift(1)
'''

"\ntest_df['alt_shift'] = test_df['alt_mean_prev3'].shift(1)\ntest_df['Mach_shift'] = test_df['Mach_mean_prev3'].shift(1)\ntest_df['TRA_shift'] = test_df['TRA_mean_prev3'].shift(1)\ntest_df['T2_shift'] = test_df['T2_mean_prev3'].shift(1)\ntest_df['T24_shift'] = test_df['T24_mean_prev3'].shift(1)\ntest_df['T30_shift'] = test_df['T30_mean_prev3'].shift(1)\ntest_df['T48_shift'] = test_df['T48_mean_prev3'].shift(1)\ntest_df['T50_shift'] = test_df['T50_mean_prev3'].shift(1)\ntest_df['P15_shift'] = test_df['P15_mean_prev3'].shift(1)\ntest_df['P2_shift'] = test_df['P2_mean_prev3'].shift(1)\ntest_df['P21_shift'] = test_df['P21_mean_prev3'].shift(1)\ntest_df['P24_shift'] = test_df['P24_mean_prev3'].shift(1)\ntest_df['Ps30_shift'] = test_df['Ps30_mean_prev3'].shift(1)\ntest_df['P40_shift']  = test_df['P40_mean_prev3'].shift(1)\ntest_df['P50_shift'] = test_df['P50_mean_prev3'].shift(1)\ntest_df['Nf_shift'] = test_df['Nf_mean_prev3'].shift(1)\ntest_df['Nc_shift'] = test_df['Nc_mean_prev3'].shift(1

In [ ]:
test_df['alt_minmax'] = test_df['alt_max_prev3'] - test_df['alt_min_prev3']
test_df['Mach_minmax'] = test_df['Mach_max_prev3'] - test_df['Mach_min_prev3']
test_df['TRA_minmax'] = test_df['TRA_max_prev3'] - test_df['TRA_min_prev3']
test_df['T2_minmax'] = test_df['T2_max_prev3'] - test_df['T2_min_prev3']
test_df['T24_minmax'] = test_df['T24_max_prev3'] - test_df['T24_min_prev3']
test_df['T30_minmax'] = test_df['T30_max_prev3'] - test_df['T30_min_prev3']
test_df['T48_minmax'] = test_df['T48_max_prev3'] - test_df['T48_min_prev3']
test_df['T50_minmax'] = test_df['T50_max_prev3'] - test_df['T50_min_prev3']
test_df['P15_minmax'] = test_df['P15_max_prev3'] - test_df['P15_min_prev3']
test_df['P2_minmax'] = test_df['P2_max_prev3'] - test_df['P2_min_prev3']
test_df['P21_minmax'] = test_df['P21_max_prev3'] - test_df['P21_min_prev3']
test_df['P24_minmax'] = test_df['P24_max_prev3'] - test_df['P24_min_prev3']
test_df['Ps30_minmax'] = test_df['Ps30_max_prev3'] - test_df['Ps30_min_prev3']
test_df['P40_minmax'] = test_df['P40_max_prev3'] - test_df['P40_min_prev3']
test_df['P50_minmax'] = test_df['P50_max_prev3'] - test_df['P50_min_prev3']
test_df['Nf_minmax'] = test_df['Nf_max_prev3'] - test_df['Nf_min_prev3']
test_df['Nc_minmax'] = test_df['Nc_max_prev3'] - test_df['Nc_min_prev3']
test_df['Wf_minmax'] = test_df['Wf_max_prev3'] - test_df['Wf_min_prev3']
test_df['T40_minmax'] = test_df['T40_max_prev3'] - test_df['T40_min_prev3']
test_df['P30_minmax'] = test_df['P30_max_prev3'] - test_df['P30_min_prev3']
test_df['P45_minmax'] = test_df['P45_max_prev3'] - test_df['P45_min_prev3']
test_df['W21_minmax'] = test_df['W21_max_prev3'] - test_df['W21_min_prev3']
test_df['W22_minmax'] = test_df['W22_max_prev3'] - test_df['W22_min_prev3']
test_df['W25_minmax'] = test_df['W25_max_prev3'] - test_df['W25_min_prev3']
test_df['W31_minmax'] = test_df['W31_max_prev3'] - test_df['W31_min_prev3']
test_df['W32_minmax'] = test_df['W32_max_prev3'] - test_df['W32_min_prev3']
test_df['W48_minmax'] = test_df['W48_max_prev3'] - test_df['W48_min_prev3']
test_df['W50_minmax'] = test_df['W50_max_prev3'] - test_df['W50_min_prev3']
test_df['SmFan_minmax'] = test_df['SmFan_max_prev3'] - test_df['SmFan_min_prev3']
test_df['SmLPC_minmax'] = test_df['SmLPC_max_prev3'] - test_df['SmLPC_min_prev3']
test_df['SmHPC_minmax'] = test_df['SmHPC_max_prev3'] - test_df['SmHPC_min_prev3']
test_df['phi_minmax'] = test_df['phi_max_prev3'] - test_df['phi_min_prev3']
test_df['fan_eff_minmax'] = test_df['fan_eff_mod_max_prev3'] - test_df['fan_eff_mod_min_prev3']
test_df['fan_flow_minmax'] = test_df['fan_flow_mod_max_prev3'] - test_df['fan_flow_mod_min_prev3']
test_df['LPC_eff_minmax'] = test_df['LPC_eff_mod_max_prev3'] - test_df['LPC_eff_mod_min_prev3']
test_df['LPC_flow_minmax'] = test_df['LPC_flow_mod_max_prev3'] - test_df['LPC_flow_mod_min_prev3']
test_df['HPC_eff_minmax'] = test_df['HPC_eff_mod_max_prev3'] - test_df['HPC_eff_mod_min_prev3']
test_df['HPC_flow_minmax'] = test_df['HPC_flow_mod_max_prev3'] - test_df['HPC_flow_mod_min_prev3']
test_df['HPT_eff_minmax'] = test_df['HPT_eff_mod_max_prev3'] - test_df['HPT_eff_mod_min_prev3']
test_df['HPT_flow_minmax'] = test_df['HPT_flow_mod_max_prev3'] - test_df['HPT_flow_mod_min_prev3']
test_df['LPT_eff_minmax'] = test_df['LPT_eff_mod_max_prev3'] - test_df['LPT_eff_mod_min_prev3']
test_df['LPT_flow_minmax'] = test_df['LPT_flow_mod_max_prev3'] - test_df['LPT_flow_mod_min_prev3']
test_df['Fc_minmax'] = test_df['Fc_max_prev3'] - test_df['Fc_min_prev3']
test_df['hs_minmax'] = test_df['hs_max_prev3'] - test_df['hs_min_prev3']

In [ ]:
df['cycle'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

In [ ]:
#test_df.fillna(method = 'bfill')

In [ ]:
test_df['cycle'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40])

学習・評価

In [ ]:
drop_features += [id, target,'unit','filename-unit','margin','margin_HPT']
features = [col for col in df.columns if col not in drop_features]
cat_features = []

In [ ]:
tmp_df = df[['filename', 'filename-unit']].drop_duplicates()
tmp_train_df, tmp_val_df = train_test_split(tmp_df, test_size=0.2, random_state=1234, stratify=tmp_df[['filename']]) # ファイルごとにunitがバランス良くtrain, testに分かれるようにする
tmp_train_df.sort_values('filename-unit', inplace=True)
tmp_val_df.sort_values('filename-unit', inplace=True)
train_fileunits = tmp_train_df['filename-unit'].values
val_fileunits = tmp_val_df['filename-unit'].values
train_fileunits, val_fileunits

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(array(['DS01-005-01', 'DS01-005-03', 'DS01-005-04', 'DS01-005-05',
        'DS02-006-02', 'DS02-006-05', 'DS02-006-10', 'DS02-006-18',
        'DS03-012-01', 'DS03-012-02', 'DS03-012-04', 'DS03-012-05',
        'DS03-012-06', 'DS03-012-08', 'DS04-02', 'DS04-03', 'DS04-04',
        'DS04-05', 'DS05-01', 'DS05-02', 'DS05-04', 'DS05-05', 'DS06-01',
        'DS06-02', 'DS06-04', 'DS06-05', 'DS07-01', 'DS07-02', 'DS07-03',
        'DS07-05', 'DS08a-009-02', 'DS08a-009-03', 'DS08a-009-05',
        'DS08a-009-06', 'DS08a-009-07', 'DS08a-009-08', 'DS08c-008-01',
        'DS08c-008-02', 'DS08c-008-03', 'DS08c-008-05'], dtype=object),
 array(['DS01-005-02', 'DS02-006-16', 'DS03-012-03', 'DS03-012-07',
        'DS04-01', 'DS05-03', 'DS06-03', 'DS07-04', 'DS08a-009-01',
        'DS08a-009-04', 'DS08c-008-04'], dtype=object))

In [ ]:
train_df = df[df['filename-unit'].isin(train_fileunits)]
val_df = df[df['filename-unit'].isin(val_fileunits)]
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
train_df.shape, val_df.shape

((3072, 1004), (772, 1004))

In [ ]:

train_X, train_y = train_df[features], train_df[target]
val_X, val_y = val_df[features], val_df[target]
X_test = test_df[features]

#train_X = train_X.apply(lambda x: x.fillna(x.mean()),axis=0)
#val_X = val_X.apply(lambda x: x.fillna(x.mean()),axis=0)
#X_test = X_test.apply(lambda x: x.fillna(x.mean()),axis=0)

train_X = train_X.to_numpy()
train_y = train_y.to_numpy().reshape(-1,1)
val_X = val_X.to_numpy()
val_y = val_y.to_numpy().reshape(-1,1)
X_test = X_test.to_numpy()

'''
#ランクガウス
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(n_quantiles=100,output_distribution='normal')
qt.fit(train_X)
train_X = qt.transform(train_X)
val_X = qt.transform(val_X)
X_test = qt.transform(X_test)
'''
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
mm.fit(train_X)
train_X = mm.transform(train_X)
val_X = mm.transform(val_X)
X_test = mm.transform(X_test)

'''

#標準化
from sklearn import preprocessing
sc = preprocessing.StandardScaler()
sc.fit(train_X)
train_X = sc.transform(train_X)
val_X = sc.transform(val_X)
X_test = sc.transform(X_test)
'''

"\n#ランクガウス\nfrom sklearn.preprocessing import QuantileTransformer\nqt = QuantileTransformer(n_quantiles=100,output_distribution='normal')\nqt.fit(train_X)\ntrain_X = qt.transform(train_X)\nval_X = qt.transform(val_X)\nX_test = qt.transform(X_test)\n\nfrom sklearn.preprocessing import MinMaxScaler\nmm = MinMaxScaler()\nmm.fit(train_X)\ntrain_X = mm.transform(train_X)\nval_X = mm.transform(val_X)\nX_test = mm.transform(X_test)\n\n\n\n#標準化\nfrom sklearn import preprocessing\nsc = preprocessing.StandardScaler()\nsc.fit(train_X)\ntrain_X = sc.transform(train_X)\nval_X = sc.transform(val_X)\nX_test = sc.transform(X_test)\n"

In [ ]:
'''
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_X.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop()

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()
'''

In [ ]:
'''
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
mm.fit(train_X)
sc_train_X = mm.transform(train_X)
sc_val_X = mm.transform(val_X)
sc_X_test = mm.transform(X_test)
'''

In [ ]:
'''
EPOCHS = 1000

history = model.fit(
  sc_train_X, train_y,
  epochs=EPOCHS, verbose=0,
  validation_data = (sc_val_X,val_y),
  )
'''

In [ ]:
#test_predictions = model.predict(sc_X_test)

In [ ]:

from pytorch_tabnet.tab_model import TabNetRegressor
import torch

regressor = TabNetRegressor(verbose=0,seed=42,n_steps=3,
                       gamma=1.0,
                       lambda_sparse=1e-4,
                       momentum=0.3, optimizer_fn=torch.optim.Adam,
                       )
regressor.fit(X_train=train_X, y_train=train_y,
              eval_set=[(val_X, val_y)],
              patience=100, max_epochs=1000,
              eval_metric=['rmse'])
predictions = regressor.predict(X_test)


In [ ]:
#pseudo labelingやりたい 7.35951 momentum 0.3 7.34694 momentum 0.4

In [ ]:
'''
params = {
    "objective": "regression_l1",
    "boosting": "gbdt",
    "metric": "mae",
    "max_depth": -1,
    "min_data_in_leaf": 250,
    "min_gain_to_split": 0.1,
    "reg_alpha": 0.1,
    "reg_lambda": 1,
    "num_leaves": 33,
    "max_bin": 256,
    "learning_rate": 0.05,
    "subsample": 1,
    "colsample_bytree": 0.8,
    "verbose": -1,
}

train_x, train_y = train_df[features], train_df[target]
val_x, val_y = val_df[features], val_df[target]
test_x = test_df[features]

dtrain = lgb.Dataset(train_x, label=train_y, categorical_feature=cat_features)
dval = lgb.Dataset(val_x, label=val_y, reference=dtrain, categorical_feature=cat_features)
bst = lgb.train(
    params, dtrain, num_boost_round=10000,
    valid_sets=[dtrain, dval], early_stopping_rounds=1000, verbose_eval=1000,)
'''

In [ ]:
'''
lgb.plot_importance(bst, figsize=(12, 12), importance_type='gain', max_num_features=50)
plt.tight_layout()
plt.show()
'''

In [ ]:
'''
pred = bst.predict(val_x, num_iteration=bst.best_iteration)
val_df[f'pred_{target}'] = pred
val_df[['unit-cycle', 'Y', 'filename', 'pred_Y']].head()
'''

unit-cycleごとのYを予測できましたが、最終的に求めたいのはunitごとのRULです。<br>
今回は、予測値に対して最もRMSEが小さくなるRULを求めることとします。

In [ ]:
'''
# unitが終わるindexを特定

unit_end_idxs = [(0, val_df.at[0,'filename-unit'])]

for i, uc in enumerate(val_df['filename-unit'].values):
    if uc != unit_end_idxs[-1][1]:
        unit_end_idxs.append((i,uc))
    if i == val_df.shape[0]-1:
        unit_end_idxs.append((i+1,''))

unit_end_idxs
'''

In [ ]:
def calc_optim_rul(pred):
    """最もRMSEが小さくなるRULを返す"""
    min_rmse_rul = 100
    min_rmse = 1e9

    for rul in range(100, 100-len(pred), -1):
        tmp_pred = [i if i > 0 else 0 for i in range(rul, rul-len(pred), -1)]
        rmse = np.sqrt(mean_squared_error(tmp_pred, pred))
        if rmse < min_rmse:
            min_rmse_rul = rul
            min_rmse = rmse

    return min_rmse_rul

In [ ]:
'''
# cycle1の時点のrulを100から1まで動かし、予測値とのrmseが最も小さい値を初期値とみなす

adj_pred = []

for i in range(1, len(unit_end_idxs)):
    start_idx = unit_end_idxs[i-1][0]
    end_idx = unit_end_idxs[i][0]
    unit_pred = pred[start_idx:end_idx]
    min_rmse_rul = calc_optim_rul(unit_pred)
    adj_unit_pred = [j if j > 0 else 0 for j in range(min_rmse_rul, min_rmse_rul - len(unit_pred), -1)]
    adj_pred += adj_unit_pred

assert len(pred) == len(adj_pred)
'''

In [ ]:
#val_df[f'postprocess_pred_{target}'] = adj_pred

In [ ]:
'''
filenames = list(val_df['filename'].unique())

for filename in filenames[:5]:
    tmp_df = val_df[val_df['filename'] == filename]
    units = list(tmp_df['unit'].unique())
    for unit in units:
        tmp_df2 = tmp_df[tmp_df['unit'] == unit]
        tmp_df2[[target, f'pred_{target}', f'postprocess_pred_{target}']].plot(figsize=(12,6), title=f'{filename}_unit{unit}')
        plt.show()
'''

テストデータに対する予測

In [ ]:
#pred = bst.predict(test_x, num_iteration=bst.best_iteration)

In [ ]:
# unitが終わるindexを特定

unit_end_idxs = [(0, test_df.at[0,'filename-unit'])]

for i, uc in enumerate(test_df['filename-unit'].values):
    if uc != unit_end_idxs[-1][1]:
        unit_end_idxs.append((i,uc))
    if i == test_df.shape[0]-1:
        unit_end_idxs.append((i+1,''))

unit_end_idxs

[(0, 'DS01-005-06'),
 (40, 'DS01-005-07'),
 (80, 'DS01-005-08'),
 (120, 'DS01-005-09'),
 (160, 'DS01-005-10'),
 (200, 'DS02-006-11'),
 (240, 'DS02-006-14'),
 (280, 'DS02-006-15'),
 (320, 'DS02-006-20'),
 (360, 'DS03-012-09'),
 (400, 'DS03-012-10'),
 (440, 'DS03-012-11'),
 (480, 'DS03-012-12'),
 (520, 'DS03-012-13'),
 (560, 'DS03-012-14'),
 (600, 'DS03-012-15'),
 (640, 'DS04-06'),
 (680, 'DS04-07'),
 (720, 'DS04-08'),
 (760, 'DS04-09'),
 (800, 'DS04-10'),
 (840, 'DS05-06'),
 (880, 'DS05-07'),
 (920, 'DS05-08'),
 (960, 'DS05-09'),
 (1000, 'DS05-10'),
 (1040, 'DS06-06'),
 (1080, 'DS06-07'),
 (1120, 'DS06-08'),
 (1160, 'DS06-09'),
 (1200, 'DS06-10'),
 (1240, 'DS07-06'),
 (1280, 'DS07-07'),
 (1320, 'DS07-08'),
 (1360, 'DS07-09'),
 (1400, 'DS07-10'),
 (1440, 'DS08a-009-09'),
 (1480, 'DS08a-009-10'),
 (1520, 'DS08a-009-11'),
 (1560, 'DS08a-009-12'),
 (1600, 'DS08a-009-13'),
 (1640, 'DS08a-009-14'),
 (1680, 'DS08a-009-15'),
 (1720, 'DS08c-008-06'),
 (1760, 'DS08c-008-07'),
 (1800, 'DS08c-008-0

In [ ]:
# cycle1の時点のrulを100から1まで動かし、予測値とのrmseが最も小さい値を初期値とみなす

pred_data = []

for i in range(1, len(unit_end_idxs)):
    start_idx = unit_end_idxs[i-1][0]
    end_idx = unit_end_idxs[i][0]
    unit_pred = predictions[start_idx:end_idx]
    min_rmse_rul = calc_optim_rul(unit_pred)
    filename, unit = unit_end_idxs[i-1][1][:-3], int(unit_end_idxs[i-1][1][-2:])
    pred_data.append([filename, unit, min_rmse_rul])

In [ ]:
pred_df = pd.DataFrame(pred_data, columns=['filename', 'unit', 'Y'])
pred_df.head()

,filename,unit,Y
0,DS01-005,6,83
1,DS01-005,7,97
2,DS01-005,8,88
3,DS01-005,9,97
4,DS01-005,10,77


In [ ]:
sub_df = pd.merge(sub_df[['filename', 'unit']], pred_df, on=['filename', 'unit'])
print(sub_df.shape)
sub_df.head()

(48, 3)


,filename,unit,Y
0,DS01-005,6,83
1,DS01-005,7,97
2,DS01-005,8,88
3,DS01-005,9,97
4,DS01-005,10,77


In [ ]:
sub_df.to_csv('./drive/MyDrive/engine/sub.csv',index=False)

In [ ]:
print(sub_df)

     filename  unit   Y
0    DS01-005     6  83
1    DS01-005     7  97
2    DS01-005     8  88
3    DS01-005     9  97
4    DS01-005    10  77
5    DS02-006    11  74
6    DS02-006    14  95
7    DS02-006    15  81
8    DS02-006    20  74
9    DS03-012     9  87
10   DS03-012    10  72
11   DS03-012    11  70
12   DS03-012    12  86
13   DS03-012    13  70
14   DS03-012    14  87
15   DS03-012    15  72
16       DS04     6  80
17       DS04     7  94
18       DS04     8  93
19       DS04     9  89
20       DS04    10  83
21       DS05     6  74
22       DS05     7  82
23       DS05     8  95
24       DS05     9  75
25       DS05    10  92
26       DS06     6  74
27       DS06     7  81
28       DS06     8  93
29       DS06     9  74
30       DS06    10  90
31       DS07     6  74
32       DS07     7  71
33       DS07     8  88
34       DS07     9  73
35       DS07    10  86
36  DS08a-009     9  74
37  DS08a-009    10  72
38  DS08a-009    11  64
39  DS08a-009    12  61
40  DS08a-009   